In [1]:
from scheduler.core.solve import solve_schedule
from scheduler.core.domain import ScheduleRequest, Worker
from scheduler.interface.utils import (
    result_to_df,
    schedule_pivot,
    hours_per_worker,
    day_worker_matrix,
)
from scheduler.config.settings import TURN_HOURS
from scheduler.core.restrictions_manager import ACTIVE_RESTRICTIONS

In [2]:
workers = [
    Worker(id=0, name="Ana", max_hours=40),
    Worker(id=1, name="Luis", max_hours=40),
    Worker(id=2, name="Marta", max_hours=40),
    Worker(id=3, name="Juan", max_hours=40),
    Worker(id=4, name="Antonio", max_hours=40),
    Worker(id=5, name="Pepe", max_hours=40),
    Worker(id=6, name="Fran", max_hours=40),
    Worker(id=7, name="Sofía", max_hours=40),
    # Worker(id=8, name="Sofía", max_hours=40),
    # Worker(id=9, name="Marta", max_hours=40),
    # Worker(id=10, name="Juan", max_hours=40),
    # Worker(id=11, name="Sofía", max_hours=40),
]

days = list(range(7))
shifts = list(range(4))

# Disponibilidad
availability = {w.id: {d: {t: 1 for t in shifts} for d in days} for w in workers}

# Ejemplo: Juan no disponible día 3 completo
for t in shifts:
    availability[3][3][t] = 0

# Demanda baja (ejemplo ideal)
demand = {d: {t: 3 if t!=2 else 2 for t in shifts} for d in days}

request = ScheduleRequest(workers, availability, demand)
request

In [3]:
ACTIVE_RESTRICTIONS_2 = ACTIVE_RESTRICTIONS.copy()

In [4]:
ACTIVE_RESTRICTIONS_2.pop(-1)

<function scheduler.core.restrictions.rest2days.add_rest2days_constraints(model, variables, schedule_request)>

In [5]:
result = solve_schedule(request, restrictions=ACTIVE_RESTRICTIONS)
print("Estado:", result["status"])


Estado: Optimal


In [6]:
model = result["model"]

In [7]:
model


Horario_Camareros_Soft:
MINIMIZE
100.0*deficit_0_0 + 100.0*deficit_0_1 + 100.0*deficit_0_2 + 100.0*deficit_0_3 + 100.0*deficit_1_0 + 100.0*deficit_1_1 + 100.0*deficit_1_2 + 100.0*deficit_1_3 + 100.0*deficit_2_0 + 100.0*deficit_2_1 + 100.0*deficit_2_2 + 100.0*deficit_2_3 + 100.0*deficit_3_0 + 100.0*deficit_3_1 + 100.0*deficit_3_2 + 100.0*deficit_3_3 + 100.0*deficit_4_0 + 100.0*deficit_4_1 + 100.0*deficit_4_2 + 100.0*deficit_4_3 + 100.0*deficit_5_0 + 100.0*deficit_5_1 + 100.0*deficit_5_2 + 100.0*deficit_5_3 + 100.0*deficit_6_0 + 100.0*deficit_6_1 + 100.0*deficit_6_2 + 100.0*deficit_6_3 + 100*empty_0_0 + 100*empty_0_1 + 100*empty_0_2 + 100*empty_0_3 + 100*empty_1_0 + 100*empty_1_1 + 100*empty_1_2 + 100*empty_1_3 + 100*empty_2_0 + 100*empty_2_1 + 100*empty_2_2 + 100*empty_2_3 + 100*empty_3_0 + 100*empty_3_1 + 100*empty_3_2 + 100*empty_3_3 + 100*empty_4_0 + 100*empty_4_1 + 100*empty_4_2 + 100*empty_4_3 + 100*empty_5_0 + 100*empty_5_1 + 100*empty_5_2 + 100*empty_5_3 + 100*empty_6_0 + 100*emp

In [8]:
(
    x,
    deficit,
    y_full,
    y_split_MT,
    y_split_MN,
    y_split_MnN,
    z,
    work,
    free2,
    viol_rest,
    viol_max,
) = result["variables"].values()

print("=== viol_rest ===")
for wid, var in viol_rest.items():
    print(wid, var.value())

print("\n=== free2 ===")
for (wid, d), var in free2.items():
    print(wid, d, var.value())

print("\n=== work ===")
for (wid, d), var in work.items():
    print(wid, d, var.value())


=== viol_rest ===
0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0

=== free2 ===
0 0 1.0
0 1 0.0
0 2 0.0
0 3 0.0
0 4 0.0
0 5 0.0
0 6 0.0
1 0 0.0
1 1 0.0
1 2 1.0
1 3 0.0
1 4 0.0
1 5 0.0
1 6 0.0
2 0 0.0
2 1 0.0
2 2 0.0
2 3 0.0
2 4 0.0
2 5 0.0
2 6 1.0
3 0 0.0
3 1 0.0
3 2 0.0
3 3 1.0
3 4 0.0
3 5 0.0
3 6 0.0
4 0 0.0
4 1 0.0
4 2 0.0
4 3 0.0
4 4 0.0
4 5 0.0
4 6 1.0
5 0 0.0
5 1 0.0
5 2 0.0
5 3 0.0
5 4 1.0
5 5 0.0
5 6 0.0
6 0 0.0
6 1 0.0
6 2 0.0
6 3 0.0
6 4 1.0
6 5 0.0
6 6 0.0
7 0 0.0
7 1 1.0
7 2 0.0
7 3 0.0
7 4 0.0
7 5 0.0
7 6 0.0

=== work ===
0 0 0.0
0 1 0.0
0 2 1.0
0 3 1.0
0 4 1.0
0 5 1.0
0 6 1.0
1 0 1.0
1 1 1.0
1 2 0.0
1 3 0.0
1 4 1.0
1 5 1.0
1 6 1.0
2 0 0.0
2 1 1.0
2 2 1.0
2 3 1.0
2 4 1.0
2 5 1.0
2 6 0.0
3 0 1.0
3 1 1.0
3 2 1.0
3 3 0.0
3 4 0.0
3 5 1.0
3 6 1.0
4 0 0.0
4 1 1.0
4 2 1.0
4 3 1.0
4 4 1.0
4 5 1.0
4 6 0.0
5 0 1.0
5 1 1.0
5 2 1.0
5 3 1.0
5 4 0.0
5 5 0.0
5 6 1.0
6 0 1.0
6 1 1.0
6 2 1.0
6 3 1.0
6 4 0.0
6 5 0.0
6 6 1.0
7 0 1.0
7 1 0.0
7 2 0.0
7 3 1.0
7 4 1.0
7 5 1.0
7 6 1.0


In [9]:
print("=== x ===")
for wid, var in x.items():
    print(wid, var.value())

=== x ===
(0, 0, 0) 0.0
(0, 0, 1) 0.0
(0, 0, 2) 0.0
(0, 0, 3) 0.0
(0, 1, 0) 0.0
(0, 1, 1) 0.0
(0, 1, 2) 0.0
(0, 1, 3) 0.0
(0, 2, 0) 1.0
(0, 2, 1) 1.0
(0, 2, 2) 1.0
(0, 2, 3) 0.0
(0, 3, 0) 1.0
(0, 3, 1) 1.0
(0, 3, 2) 0.0
(0, 3, 3) 0.0
(0, 4, 0) 1.0
(0, 4, 1) 0.0
(0, 4, 2) 0.0
(0, 4, 3) 1.0
(0, 5, 0) 0.0
(0, 5, 1) 1.0
(0, 5, 2) 1.0
(0, 5, 3) 1.0
(0, 6, 0) 0.0
(0, 6, 1) 0.0
(0, 6, 2) 0.0
(0, 6, 3) 1.0
(1, 0, 0) 0.0
(1, 0, 1) 0.0
(1, 0, 2) 1.0
(1, 0, 3) 1.0
(1, 1, 0) 0.0
(1, 1, 1) 1.0
(1, 1, 2) 1.0
(1, 1, 3) 1.0
(1, 2, 0) 0.0
(1, 2, 1) 0.0
(1, 2, 2) 0.0
(1, 2, 3) 0.0
(1, 3, 0) 0.0
(1, 3, 1) 0.0
(1, 3, 2) 0.0
(1, 3, 3) 0.0
(1, 4, 0) 1.0
(1, 4, 1) 1.0
(1, 4, 2) 0.0
(1, 4, 3) 0.0
(1, 5, 0) 1.0
(1, 5, 1) 1.0
(1, 5, 2) 0.0
(1, 5, 3) 0.0
(1, 6, 0) 1.0
(1, 6, 1) 1.0
(1, 6, 2) 0.0
(1, 6, 3) 0.0
(2, 0, 0) 0.0
(2, 0, 1) 0.0
(2, 0, 2) 0.0
(2, 0, 3) 0.0
(2, 1, 0) 0.0
(2, 1, 1) 0.0
(2, 1, 2) 0.0
(2, 1, 3) 1.0
(2, 2, 0) 0.0
(2, 2, 1) 0.0
(2, 2, 2) 0.0
(2, 2, 3) 1.0
(2, 3, 0) 0.0
(2, 3, 1) 1.0
(2, 3, 2) 

In [11]:
df_sched = result_to_df(result, request)
pivot = schedule_pivot(df_sched)
pivot


Turno,0,1,2,3
Día,,,,
0,"Juan, Pepe, Fran","Juan, Pepe, Fran, Sofía","Luis, Pepe, Sofía","Luis, Pepe, Sofía"
1,"Juan, Antonio, Fran","Luis, Juan, Antonio, Pepe","Luis, Pepe","Luis, Marta, Pepe"
2,"Ana, Antonio, Fran","Ana, Juan, Antonio, Fran","Ana, Juan, Antonio","Marta, Juan, Pepe"
3,"Ana, Antonio, Sofía","Ana, Marta, Antonio, Fran, Sofía","Marta, Fran","Marta, Pepe, Fran"
4,"Ana, Luis, Sofía","Luis, Marta, Antonio, Sofía","Marta, Antonio","Ana, Marta, Antonio"
5,"Luis, Juan, Sofía","Ana, Luis, Marta, Juan, Sofía","Ana, Marta","Ana, Marta, Antonio"
6,"Luis, Juan, Pepe","Luis, Juan, Pepe, Fran","Fran, Sofía","Ana, Fran, Sofía"


In [12]:
hours_per_worker(df_sched, TURN_HOURS)


,Trabajador,Horas
0,Ana,40
1,Antonio,40
2,Fran,40
4,Luis,40
7,Sofía,40
3,Juan,39
5,Marta,39
6,Pepe,39


In [12]:
day_worker_matrix(df_sched)


Turno,Día,Trabajador,0,1,2,3
0,0,Ana,Sí,No,No,No
1,0,Antonio,No,Sí,Sí,Sí
2,0,Fran,No,No,No,Sí
3,0,Luis,No,Sí,Sí,Sí
4,0,Pepe,Sí,Sí,No,No
5,1,Ana,Sí,Sí,No,No
6,1,Antonio,No,Sí,No,No
7,1,Fran,No,Sí,Sí,No
8,1,Juan,Sí,Sí,No,No
9,1,Marta,No,No,Sí,Sí
